<a href="https://colab.research.google.com/github/Vladzimir-40/Proj_1/blob/main/my_file4/dz_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub

from google.colab import files
files.upload() # Загрузите файл kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vladimir401011","key":"e6b9eec8a00a0a7c2c3e5fcbd033ab06"}'}

In [4]:

import pandas as pd
import kagglehub
import os

import pandas as pd
import kagglehub
import os

class Data_loading:
    @staticmethod
    def load_from_kaggle(dataset_path, filename, file_type='csv'):
        """
        Загружает dataset из Kaggle используя kagglehub

        Args:
            dataset_path (str): путь к dataset на Kaggle
            filename (str): название файла для загрузки
            file_type (str): тип файла - 'csv' или 'json'

        Returns:
            pd.DataFrame: загруженный датафрейм
        """
        download_path = kagglehub.dataset_download(dataset_path)
        file_path = os.path.join(download_path, filename)

        if file_type == 'csv':
            return pd.read_csv(file_path)
        elif file_type == 'json':
            return pd.read_json(file_path)
        else:
            raise ValueError("Поддерживаются только 'csv' и 'json' форматы")


import kagglehub

# Download latest version
path = kagglehub.dataset_download("kaggle/recipe-ingredients-dataset")

print("Path to dataset files:", path)
print("\nЗагруженные файлы:")
for file in os.listdir(path):
    print(f"📁 {file}")

100%|██████████| 2.23M/2.23M [00:00<00:00, 130MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/kaggle/recipe-ingredients-dataset/versions/1

Загруженные файлы:
📁 train.json
📁 test.json


Так мы видим, что данный датасет пердставлен двумя json-файлами.               
"Обучающий" train.json - содердит поле "категирию" (содержит порядка 40000 строк)         
А "Тестовый" test.json - нужен для проверки обученных моделей (содержит порядка 10000)

In [5]:
# Загрузка обучающего датасета
df_train = Data_loading.load_from_kaggle("kaggle/recipe-ingredients-dataset", "train.json", "json")

# Загрузка тестового датасета
df_test = Data_loading.load_from_kaggle("kaggle/recipe-ingredients-dataset", "test.json", "json")

Using Colab cache for faster access to the 'recipe-ingredients-dataset' dataset.
Using Colab cache for faster access to the 'recipe-ingredients-dataset' dataset.


In [7]:


# Просматриваем первые несколько строк датасета
print(df_train.head())

# Получаем общую информацию о датасете
print(df_train.info())

      id      cuisine                                        ingredients
0  10259        greek  [romaine lettuce, black olives, grape tomatoes...
1  25693  southern_us  [plain flour, ground pepper, salt, tomatoes, g...
2  20130     filipino  [eggs, pepper, salt, mayonaise, cooking oil, g...
3  22213       indian                [water, vegetable oil, wheat, salt]
4  13162       indian  [black pepper, shallots, cornflour, cayenne pe...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39774 entries, 0 to 39773
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           39774 non-null  int64 
 1   cuisine      39774 non-null  object
 2   ingredients  39774 non-null  object
dtypes: int64(1), object(2)
memory usage: 932.3+ KB
None


Для начала, 40000 строк - это многовато. Для обучения хватит и 3000.

По возможности отберем классы (категории или кухни) пропорционально тому, как они представлены в датасете

In [8]:
# Стратифицированный отбор (сохраняет пропорции классов)
from sklearn.model_selection import train_test_split

# Функция возвращает две переменные. Функция делит датасет на 3000 отобраных строк
# (это так называемая "тестовая" выбока) и всё остальное. Так как нам нужно
# нужна только первая перемееная, то на второй переменной мы ставим знак нижнего
# подчеркивания, и эта переменная, по правилам синтакиса в Python, использоваться
# не будет
df_limited, _ = train_test_split(df_train, train_size=3000,
                                stratify=df_train['cuisine'], random_state=42)

# Проверяем: дейстительно 3000 строк
print(df_limited.info())

<class 'pandas.core.frame.DataFrame'>
Index: 3000 entries, 33136 to 23167
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           3000 non-null   int64 
 1   cuisine      3000 non-null   object
 2   ingredients  3000 non-null   object
dtypes: int64(1), object(2)
memory usage: 93.8+ KB
None


Вторая проблема с данными в том, что ингредиенты записаны в матрицы. Мы трижды преобразуем их следующим образом:
1) Перобразуем матрицу в строку
2) Преобзазуем строку в вектор-словарь [значение:вес] (функция "fit")
3) Преобзуем все строки с векторами в одну общую матрицу (функция "transform")

In [10]:
# Преобразуй списки в текст:
df_limited['ingredients_text'] = df_limited['ingredients'].apply(lambda x: ' '.join(x))

# Векторизация: TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Указываем оставить только  1000 самых частых ингредиентов (max_features=1000)
# Считается, что редкие ингредиенты мало влияют на классификацию !!!
# Удаляем английские союзы, как возможный шум (stop_words='english')
tfidf = TfidfVectorizer(max_features=1000, stop_words='english')
X = tfidf.fit_transform(df_limited['ingredients_text'])

# Целевая переменная:
y = df_limited['cuisine']

Мы имеем многоклассовую классификацию (несколько типов кухонь). Наши данные после преобразования - это разреженная матрица (sparse matrix) с 1000 признаками (ингредиенты) и 3000 образцов.

Попробуем сравнить так: Бустовые модели протрив деревянных три на две

Бустовые:
  * CatBoost
  * Классификатор градиентного бустинга
  * Light Gradient Boosting Machine

Деревянные:
  * Классификатор Extra Trees
  * Классификатор дерева решений